## Create a Z Stack of images

This script creates a z stack as a TIFF file with OME META data. As sole information, the z position is included. 

As a small addition, this script also calculates the z plane of highest sharpness and set this to the 0 position, allowing for EXACT to set it as default frame.

The files in this example had the following file names:
- zyto_x7489.216_y13279.552_z6.50.png
- zyto_x7489.216_y13279.552_z7.50.png
- zyto_x7489.216_y13279.552_z8.00.png

The format hence was:

zyto_x{x}_y{y}_z{z}.png with x,y, and z being the relative coordinates of the microscope stage.



In [ ]:
import tifffile
import os
from PIL import Image
import numpy as np
import cv2

def image_sharpness(image):
    """Computes the sharpness of an image.

    This function computes the sharpness of an image based on the magnitude of
    the gradient.

    Args:
        image: A cv2 image.

    Returns:
        The average sharpness of the image.
    """
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert the image from RGB to BGR
    gray_image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)  # Convert the image to grayscale
    grad_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = cv2.addWeighted(np.absolute(grad_x), 0.5, np.absolute(grad_y), 0.5, 0)
    sharpness = np.average(gradient_magnitude)
    return sharpness


import pyvips
basepath='./'
folder='x7489.233_y13279.560'

# load all images of the stack
ims=[]
z_positions=[]
filelist={}
for file in os.listdir(basepath+folder):
    zpos = float(file.split('_')[-1][1:-4])
    z_positions.append(zpos)
    filelist[zpos]=file

# We want to sort the z positions, because the file list is not sorted for it.
z_positions=sorted(filelist)

for file in [filelist[x] for x in sorted(filelist)]:
    ims.append(np.array(Image.open(basepath+folder+os.sep+file)))
data = np.stack(ims,0)

# find image with highest sharpness (z index=0)
sharpness = [image_sharpness(x) for x in ims]

# zero Z by sharpest image
z_offset = z_positions[np.argmax(sharpness)]


with tifffile.TiffWriter(basepath+folder+'.ome.tiff', ome=True) as writer:
    for frame, zpos in enumerate(z_positions):
        ome_metadata = {'axes': 'YXC', 
                        'Plane': {'PositionZ': zpos-z_offset, 'PositionZUnit': 'µm'}}
        
        writer.write(ims[frame], metadata=ome_metadata, compression='lzw')                


